In [1]:
import napari
from matplotlib import pyplot as plt
import numpy as np
import cv2
from functions import matrix_xy_swap

In [2]:
# Open an image
img = plt.imread('test_photos/image_5.jpg')

# Open it in napari viewer
viewer = napari.view_image(img, rgb=True)

corners = np.array([
    [54.90553028, 253.60130126], # top left
    [21.82135346, 1236.20135279], # top right
    [1062.31871443, 1292.44445338], # bottom right
    [1072.24396747, 134.49826472] # bottom left
    ], dtype=np.float32)

# Add a Points layer
points_layer = viewer.add_points(corners)


At this time, use the GUI to put one point on each corner of the quadrat. 

In [3]:
# Retreive the list of points present in the points layer
corners = points_layer.data

print(corners)
# Make sure corners is a numpy array of the float32 type.

[[  54.90553   253.6013  ]
 [  21.821354 1236.2013  ]
 [1062.3187   1292.4445  ]
 [1072.244     134.49826 ]]


Swap the coordinates you got from napari because they're in (y, x) not (x, y)

In [5]:
corners_swapped = matrix_xy_swap(corners)

print(corners)
print(corners_swapped)

corners = corners_swapped

[[  54.90553   253.6013  ]
 [  21.821354 1236.2013  ]
 [1062.3187   1292.4445  ]
 [1072.244     134.49826 ]]
[[ 253.6013     54.90553 ]
 [1236.2013     21.821354]
 [1292.4445   1062.3187  ]
 [ 134.49826  1072.244   ]]


Now plug the points into an affine transformation. 

In [6]:
# Define a side length for the square based on the top quadrat length
distance = corners[0] - corners[2] 
side_length = abs(int(distance[0])) # absolute value of difference between first & second vector

print(side_length)

1038


In [8]:
# Define the new locations of each point based on side length
corner_destinations = np.array([
    [0, 0], # top left
    [side_length, 0], # top right
    [side_length, side_length], # bottom right
    [0, side_length] # bottom left
], dtype=np.float32)

# Get perspective transformation matrix 
matrix = cv2.getPerspectiveTransform(corners, corner_destinations)

# Apply perspective transformation
img_transformed = cv2.warpPerspective(img, matrix, (side_length, side_length))

cv2.imshow('og image', img)
cv2.imshow('affine transform', img_transformed)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# Set the print options to avoid scientific notation
np.set_printoptions(suppress=True)

print(matrix)


[[   1.19036083    0.04007949  -75.52160173]
 [   0.12557823    1.07264734 -278.91970245]
 [   0.0001806     0.00003122    1.        ]]


In [39]:
print(corners)
print(corner_destinations)

[[  54.90553   253.6013  ]
 [  21.821354 1236.2013  ]
 [1062.3187   1292.4445  ]
 [1072.244     134.49826 ]]
[[   0.    0.]
 [   0. 1007.]
 [1007. 1007.]
 [1007.    0.]]


Now try one clean run with a different image

In [37]:
# Read in image
img = plt.imread('test_photos/image_21.jpg')

# Open it in napari viewer
viewer = napari.view_image(img, rgb=True)

# Add a points layer
points_layer = viewer.add_points()


Use the GUI to place points IN CLOCKWISE ORDER: top left, top right, bottom right, bottom left

In [38]:
# Retrieve points from the GUI
corners = points_layer.data

# Convert it into a np matrix
corners = np.array(corners, dtype = np.float32)

# Swap the x and y to convert from napari coordinates to python coordinates
corners_swapped = matrix_xy_swap(corners)

# Get a side length based on the top pipe of the quadrat
distance = corners_swapped[0] - corners_swapped[1]
side_length = abs(int(np.linalg.norm(distance))) # absolute value of the Euclidean distance

# Define the new locations of each point based on side length
corner_destinations = np.array([
    [0, 0], # top left
    [side_length, 0], # top right
    [side_length, side_length], # bottom right
    [0, side_length] # bottom left
], dtype=np.float32)

# Get perspective transformation matrix 
matrix = cv2.getPerspectiveTransform(corners_swapped, corner_destinations)

# Apply perspective transformation
img_transformed = cv2.warpPerspective(img, matrix, (side_length, side_length))

cv2.imshow('og image', img)
cv2.imshow('affine transform', img_transformed)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [35]:
print(corners_swapped)

[[ 212.24608    987.87933  ]
 [1391.697     1001.113    ]
 [1328.837       35.055023 ]
 [ 382.62958      3.6250563]]
